In [1]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!pip install konlpy


Installing automake (A dependency for mecab-ko)
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease

In [3]:
from collections import Counter
from konlpy.tag import *
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [25]:
corpus= [
    "나는 나를 사랑한다.",
    "나는 너를 사랑한다.",
    "너는 나를 사랑한다.",
    "너는 너를 사랑한다.",
    "나는 나를 증오한다.",
    "나는 너를 증오한다.",
    "너는 나를 증오한다.",
    "너는 너를 증오한다."
]

# 0:부정, 1:긍정, 2:모름    

df=pd.DataFrame({"id": range(len(corpus)),"corpus": corpus,"label":[0,1,1,0,2,0,0,2]})                 
class_count=len(set(df["label"]))




In [15]:

s=" ".join(df.corpus)

print('All string :' ,s)

# posToUse=["NNP","NNG","MAG","NP","VV","VV+EF","IC"]
posToUse=["NNP","NNG","NP","NNB"]

def getTokens(s):
    global posToUse
    return [ i[0] for i in  Mecab().pos(s) if i[1] in posToUse ] 
    
print('All tockents:', getTokens(s))

posToUse=["NNP","NNG","NP","NNB"]
stopwords=["놈","18"]
def getToken(s,pos=posToUse,stopword=stopwords):
    return [ w for w,t in  Mecab().pos(s) if t in pos and w not in stopword ] 
vect = CountVectorizer(tokenizer=getToken)
vect.fit(getTokens(s))
print('Dictionary : ',vect.vocabulary_)
cd=pd.DataFrame(vect.transform([" ".join(getTokens(i)) for i in df.corpus]).toarray())
data=pd.concat([df,cd], axis=1)
data

All string : 나는 나를 사랑한다. 나는 너를 사랑한다. 너는 나를 사랑한다. 너는 너를 사랑한다. 나는 나를 증오한다. 나는 너를 증오한다. 너는 나를 증오한다. 너는 너를 증오한다.
All tockents: ['나', '나', '사랑', '나', '너', '사랑', '너', '나', '사랑', '너', '너', '사랑', '나', '나', '증오', '나', '너', '증오', '너', '나', '증오', '너', '너', '증오']
Dictionary :  {'나': 0, '사랑': 2, '너': 1, '증오': 3}


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


,id,corpus,label,0,1,2,3
0,0,나는 나를 사랑한다.,0,2,0,1,0
1,1,나는 너를 사랑한다.,1,1,1,1,0
2,2,너는 나를 사랑한다.,1,1,1,1,0
3,3,너는 너를 사랑한다.,0,0,2,1,0
4,4,나는 나를 증오한다.,2,2,0,0,1
5,5,나는 너를 증오한다.,0,1,1,0,1
6,6,너는 나를 증오한다.,0,1,1,0,1
7,7,너는 너를 증오한다.,2,0,2,0,1


In [26]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop


x_train=data.iloc[:,3:11].to_numpy()
#y_train=data.label.to_numpy().reshape([-1,1])
y_train=keras.utils.to_categorical(data.label,class_count)

In [27]:
print(x_train.shape)
print(y_train.shape)

(8, 4)
(8, 3)


In [50]:
model = Sequential()
model.add(Dense(14, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dense(6, activation='relu'))
model.add(Dense(class_count, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 14)                70        
_________________________________________________________________
dense_19 (Dense)             (None, 6)                 90        
_________________________________________________________________
dense_20 (Dense)             (None, 3)                 21        
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________


In [51]:
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=100,
                    verbose=1,
                    validation_data=(x_train, y_train))


Epoch 1/100
8/8 [==============================] - 1s 25ms/step - loss: 1.3120 - accuracy: 0.4661 - val_loss: 1.1897 - val_accuracy: 0.5000
Epoch 2/100
8/8 [==============================] - 0s 7ms/step - loss: 1.1110 - accuracy: 0.7172 - val_loss: 1.1753 - val_accuracy: 0.5000
Epoch 3/100
8/8 [==============================] - 0s 6ms/step - loss: 1.2585 - accuracy: 0.5505 - val_loss: 1.1631 - val_accuracy: 0.5000
Epoch 4/100
8/8 [==============================] - 0s 27ms/step - loss: 1.1158 - accuracy: 0.6487 - val_loss: 1.1542 - val_accuracy: 0.5000
Epoch 5/100
8/8 [==============================] - 0s 6ms/step - loss: 1.1599 - accuracy: 0.3513 - val_loss: 1.1434 - val_accuracy: 0.5000
Epoch 6/100
8/8 [==============================] - 0s 7ms/step - loss: 1.2057 - accuracy: 0.3394 - val_loss: 1.1367 - val_accuracy: 0.5000
Epoch 7/100
8/8 [==============================] - 0s 6ms/step - loss: 1.0737 - accuracy: 0.6616 - val_loss: 1.1305 - val_accuracy: 0.5000
Epoch 8/100
8/8 [========

In [52]:
score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

Train loss: 0.6375794410705566
Train accuracy: 0.875


In [53]:
model.predict(x_train)

array([[0.32286388, 0.4934148 , 0.1837213 ],
       [0.38655698, 0.5163316 , 0.09711138],
       [0.38655698, 0.5163316 , 0.09711138],
       [0.6159748 , 0.17795467, 0.2060705 ],
       [0.39697734, 0.16661867, 0.436404  ],
       [0.67500633, 0.03277252, 0.29222116],
       [0.67500633, 0.03277252, 0.29222116],
       [0.41697124, 0.00509538, 0.5779334 ]], dtype=float32)

In [54]:
model.predict_classes(x_train)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([1, 1, 1, 0, 2, 0, 0, 2])